In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"

In [2]:
from crewai_tools import tool

import yfinance as yf


class Tools:

    @tool("One month stock price history")
    def stock_price(ticker):
        """
        Useful to get a month's worth of stock price data as CSV.
        The input of this tool should a ticker, for example AAPL, NET, TSLA etc...
        """
        stock = yf.Ticker(ticker)
        return stock.history(period="1mo").to_csv()

    @tool("Stock news URLs")
    def stock_news(ticker):
        """
        Useful to get URLs of news articles related to a stock.
        The input to this tool should be a ticker, for example AAPL, NET
        """
        stock = yf.Ticker(ticker)
        return list(map(lambda x: x["link"], stock.news))

    @tool("Company's income statement")
    def income_stmt(ticker):
        """
        Useful to get the income statement of a stock as CSV.
        The input to this tool should be a ticker, for example AAPL, NET
        """
        stock = yf.Ticker(ticker)
        return stock.income_stmt.to_csv()

    @tool("Balance sheet")
    def balance_sheet(ticker):
        """
        Useful to get the balance sheet of a stock as CSV.
        The input to this tool should be a ticker, for example AAPL, NET
        """
        stock = yf.Ticker(ticker)
        return stock.balance_sheet.to_csv()

    @tool("Get insider transactions")
    def insider_transactions(ticker):
        """
        Useful to get insider transactions of a stock as CSV.
        The input to this tool should be a ticker, for example AAPL, NET
        """
        stock = yf.Ticker(ticker)
        return stock.insider_transactions.to_csv()

In [3]:
from crewai import Agent


from crewai_tools import ScrapeWebsiteTool


class Agents:

    def technical_analyst(self):
        return Agent(
            role="Technical Analyst",
            goal="Analyses the movements of a stock and provides insights on trends, entry points, resistance and support levels.",
            backstory="An expert in technical analysis, you're known for your ability to predict stock movements and trends based on historical data. You provide valuable insights to your customers.",
            verbose=True,
            tools=[
                Tools.stock_price,
            ],
        )

    def researcher(self):
        return Agent(
            role="Researcher",
            goal="Gathers, interprets and summarizes vasts amounts of data to provide a comprehensive overview of the sentiment and news surrounding a stock.",
            backstory="You're skilled in gathering and interpreting data from various sources to give a complete picture of a stock's sentiment and news. You read each data source carefuly and extract the most important information. Your insights are crucial for making informed investment decisions.",
            verbose=True,
            tools=[
                Tools.stock_news,
                ScrapeWebsiteTool(),
            ],
        )

    def financial_analyst(self):
        return Agent(
            role="Financial Analyst",
            goal="Uses financial statements, insider trading data, and other financial metrics to evaluate a stock's financial health and performance.",
            backstory="You're a very experienced investment advisor who uses a combination of technical and fundamental analysis to provide strategic investment advice to your clients. You look at a company's financial health, market sentiment, and qualitative data to make informed recommendations.",
            verbose=True,
            tools=[
                Tools.balance_sheet,
                Tools.income_stmt,
                Tools.insider_transactions,
            ],
        )

    def hedge_fund_manager(self):
        return Agent(
            role="Hedge Fund Manager",
            goal="Manages a portfolio of stocks and makes strategic investment decisions to maximize returns using insights from financial analysts, technical analysts, and researchers.",
            backstory="You're a seasoned hedge fund manager with a proven track record of making profitable investment decisions. You're known for your ability to manage risk and maximize returns for your clients.",
            verbose=True,
        )

In [4]:
from crewai import Task


class Tasks:

    def research(self, agent):
        return Task(
            description="Gather and analyze the latest news and market sentiment surrounding the stock of {company}. Provide a summary of the news and any notable shifts in market sentiment.",
            expected_output=f"Your final answer MUST be a detailed summary of the news and market sentiment surrounding the stock. Include any notable shifts in market sentiment and provide insights on how these factors could impact the stock's performance.",
            agent=agent,
            output_file="stock_reports/stock_news.md",
        )

    def technical_analysis(self, agent):
        return Task(
            description="Conduct a detailed technical analysis of the price movements of {company}'s stock and trends identify key support and resistance levels, chart patterns, and other technical indicators that could influence the stock's future performance. Use historical price data and technical analysis tools to provide insights on potential entry points and price targets.",
            expected_output=f"Your final answer MUST be a detailed technical analysis report that includes key support and resistance levels, chart patterns, and technical indicators. Provide insights on potential entry points, price targets, and any other relevant information that could help your customer make informed investment decisions.",
            agent=agent,
            output_file="stock_reports/technical_analysis.md",
        )

    def finacial_analysis(self, agent):
        return Task(
            description="Analyze {company}'s financial statements, insider trading data, and other financial metrics to evaluate the stock's financial health and performance. Provide insights on the company's revenue, earnings, cash flow, and other key financial metrics. Use financial analysis tools and models to assess the stock's valuation and growth potential.",
            expected_output=f"Your final answer MUST be a detailed financial analysis report that includes insights on the company's financial health, performance, and valuation. Provide an overview of the company's revenue, earnings, cash flow, and other key financial metrics. Use financial analysis tools and models to assess the stock's valuation and growth potential.",
            agent=agent,
            output_file="stock_reports/financial_analysis.md",
        )

    def investment_recommendation(self, agent, context):
        return Task(
            description="Based on the research, technical analysis, and financial analysis reports, provide a detailed investment recommendation for {company}'s stock. Include your analysis of the stock's potential risks and rewards, and provide a clear rationale for your recommendation.",
            expected_output=f"Your final answer MUST be a detailed investment recommendation report to BUY or SELL the stock that includes your analysis of the stock's potential risks and rewards. Provide a clear rationale for your recommendation based on the research, technical analysis, and financial analysis reports.",
            agent=agent,
            context=context,
            output_file="stock_reports/investment_recommendation.md",
        )

In [5]:
from crewai import Crew
from crewai.process import Process
from langchain_openai import ChatOpenAI

agents = Agents()
tasks = Tasks()

researcher = agents.researcher()
technical_analyst = agents.technical_analyst()
financial_analyst = agents.financial_analyst()
hedge_fund_manager = agents.hedge_fund_manager()

research_task = tasks.research(researcher)
technical_task = tasks.technical_analysis(technical_analyst)
financial_task = tasks.finacial_analysis(financial_analyst)
recommend_task = tasks.investment_recommendation(
    hedge_fund_manager,
    [
        research_task,
        technical_task,
        financial_task,
    ],
)

from crewai import Crew
from crewai.process import Process
from langchain_openai import ChatOpenAI

agents = Agents()
tasks = Tasks()

researcher = agents.researcher()
technical_analyst = agents.technical_analyst()
financial_analyst = agents.financial_analyst()
hedge_fund_manager = agents.hedge_fund_manager()

research_task = tasks.research(researcher)
technical_task = tasks.technical_analysis(technical_analyst)
financial_task = tasks.finacial_analysis(financial_analyst)
recommend_task = tasks.investment_recommendation(
    hedge_fund_manager,
    [
        research_task,
        technical_task,
        financial_task,
    ],
)

crew = Crew(
    agents=[
        researcher,
        technical_analyst,
        financial_analyst,
        hedge_fund_manager,
    ],
    tasks=[
        research_task,
        technical_task,
        financial_task,
        recommend_task,
    ],
    verbose=2,
    process=Process.hierarchical,
    manager_llm=ChatOpenAI(model="gpt-4o-mini"),
    memory=True,
)

result = crew.kickoff(
    inputs=dict(
        company="apple",
    ),
)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Gather and analyze the latest news and market sentiment surrounding the stock of apple. Provide a summary of the news and any notable shifts in market sentiment.


> Entering new CrewAgentExecutor chain...
I need to gather the latest news and market sentiment regarding Apple’s stock. To do this effectively, I will first delegate the task of researching the latest news articles and sentiment analysis to the Researcher. This will ensure I get accurate and comprehensive information on Apple's current market position.

Action: Delegate work to co-worker  
Action Input: {"co-worker": "Researcher", "task": "Gather the latest news and market sentiment surrounding Apple's stock. Provide a detailed summary of the news articles and any notable shifts in market sentiment. Analyze how these factors might impact Apple's stock performance.", "context": "The goal is to understand the current landscape regarding Apple's stock. This includes

2024-08-23 10:55:13,702 - 8445201408 - sqlite.py-sqlite:284 - WARNING: Insert of existing embedding ID: default-app-id--db7339952ea3df8f4a0ef64fd63b913345479cc4bb2f50a83b2b1f31cbb2aa18


Thought: I need to gather comprehensive financial data for Apple, including their balance sheet, income statement, and insider transactions to evaluate the company's financial health and performance. This will allow me to analyze their revenue growth trends, earnings per share, cash flow from operations, and any significant insider trading activity.

Action: Balance sheet  
Action Input: {"ticker": "AAPL"}  
 

,2023-09-30,2022-09-30,2021-09-30,2020-09-30,2019-09-30
Treasury Shares Number,0.0,,,,
Ordinary Shares Number,15550061000.0,15943425000.0,16426786000.0,16976763000.0,
Share Issued,15550061000.0,15943425000.0,16426786000.0,16976763000.0,
Net Debt,81123000000.0,96423000000.0,89779000000.0,74420000000.0,
Total Debt,123930000000.0,132480000000.0,136522000000.0,122278000000.0,
Tangible Book Value,62146000000.0,50672000000.0,63090000000.0,65339000000.0,
Invested Capital,173234000000.0,170741000000.0,187809000000.0,177775000000.0,
Working Capital,-1742000000.0,-18577000000.0,9355000000

2024-08-23 10:55:17,916 - 8445201408 - sqlite.py-sqlite:284 - WARNING: Insert of existing embedding ID: default-app-id--2de4972f871acd3439a49d376ca6c606c7c301204f9d77f0002fba013f92a195


Thought: I need to gather comprehensive financial data for Apple, including their balance sheet, income statement, and insider transactions to evaluate the company's financial health and performance. This will allow me to analyze their revenue growth trends, earnings per share, cash flow from operations, and any significant insider trading activity.

Action: Get insider transactions  
Action Input: {"ticker": "AAPL"}  
 

I tried reusing the same input, I must stop using this action input. I'll try something else instead.





...

Thought: I need to gather comprehensive financial data for Apple Inc. (AAPL) to conduct a detailed financial analysis. I will start by retrieving the company's balance sheet to assess its financial health through assets and liabilities.

Action: Balance sheet  
Action Input: {"ticker": "AAPL"}  
 

,2023-09-30,2022-09-30,2021-09-30,2020-09-30,2019-09-30
Treasury Shares Number,0.0,,,,
Ordinary Shares Number,15550061000.0,15943425000.0,16426786000.0,16976763000.0,
Share Issued,15550061000.0,15943425000.0,16426786000.0,16976763000.0,
Net Debt,81123000000.0,96423000000.0,89779000000.0,74420000000.0,
Total Debt,123930000000.0,132480000000.0,136522000000.0,122278000000.0,
Tangible Book Value,62146000000.0,50672000000.0,63090000000.0,65339000000.0,
Invested Capital,173234000000.0,170741000000.0,187809000000.0,177775000000.0,
Working Capital,-1742000000.0,-18577000000.0,9355000000.0,38321000000.0,
Net Tangible Assets,62146000000.0,50672000000.0,63090000000.0,65339000000.0,
Capital Lease Obliga

2024-08-23 10:57:08,269 - 8445201408 - sqlite.py-sqlite:284 - WARNING: Insert of existing embedding ID: default-app-id--97982be412c5e4ad29829c7c71602d794ef4141f3aab8eafc4224c456cb96d9b


Thought: I have gathered the balance sheet data for Apple Inc. (AAPL). Now, I need to retrieve the company's income statement to analyze its revenue trends and earnings performance.

Action: Company's income statement  
Action Input: {"ticker": "AAPL"}  
 

,2023-09-30,2022-09-30,2021-09-30,2020-09-30
Tax Effect Of Unusual Items,0.0,0.0,0.0,0.0
Tax Rate For Calcs,0.147,0.162,0.133,0.144
Normalized EBITDA,129188000000.0,133138000000.0,123136000000.0,81020000000.0
Net Income From Continuing Operation Net Minority Interest,96995000000.0,99803000000.0,94680000000.0,57411000000.0
Reconciled Depreciation,11519000000.0,11104000000.0,11284000000.0,11056000000.0
Reconciled Cost Of Revenue,214137000000.0,223546000000.0,212981000000.0,169559000000.0
EBITDA,129188000000.0,133138000000.0,123136000000.0,81020000000.0
EBIT,117669000000.0,122034000000.0,111852000000.0,69964000000.0
Net Interest Income,-183000000.0,-106000000.0,198000000.0,890000000.0
Interest Expense,3933000000.0,2931000000.0,26450000

2024-08-23 10:57:10,392 - 8445201408 - sqlite.py-sqlite:284 - WARNING: Insert of existing embedding ID: default-app-id--dd3d04d99a5039cb9f5d5d7220ee41a6abf9c3c39cdd336a97eb955d011eb589


Thought: I have gathered the balance sheet and income statement data for Apple Inc. (AAPL). Next, I need to analyze insider trading activity to assess market sentiment and potential insider perspectives on the company's future.

Action: Get insider transactions  
Action Input: {"ticker": "AAPL"}  
 

,Shares,Value,URL,Text,Insider,Position,Transaction,Start Date,Ownership
0,8706,1958850.0,,Sale at price 225.00 per share.,KONDO CHRISTOPHER,Officer,,2024-08-15,D
1,5178,1121037.0,,Sale at price 216.50 per share.,KONDO CHRISTOPHER,Officer,,2024-08-09,D
2,4500,0.0,,Stock Gift at price 0.00 per share.,ADAMS KATHERINE L,General Counsel,,2024-08-07,D
3,100000,20643512.0,,Sale at price 206.42 - 207.05 per share.,ADAMS KATHERINE L,General Counsel,,2024-08-05,D
4,75000,14368500.0,,Sale at price 191.58 per share.,LEVINSON ARTHUR D,Director,,2024-05-30,D
5,4999,951785.0,,Sale at price 190.40 per share.,KONDO CHRISTOPHER,Officer,,2024-05-15,D
6,1850,0.0,,Stock Gift at price 0.00 per share.,ADAMS KAT